In [ ]:
import keras
from keras.models import Sequential
#from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils
import cv2
import math
import os
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

In [ ]:
train = pd.read_csv(r'D:\2020Spring\636_project\data\train_new.csv')
train.head()
#print(train)

In [ ]:
#creating an empty list
#train_image = []
#train_image_temp = []
X = []
i_count = train.shape[0]
i_count = int(i_count/15)
print(i_count)
# for loop to read and store frames
for i in tqdm(range(i_count)):
#for i in range(150):
    train_image = []
    train_image_temp = []
    for j in range(15):
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(r'D:\2020Spring\636_project\data\train_1/'+train['image'][i*15+j])
        #print(i*15+j)
        #print(train['image'][i*15+j])
        # converting it to array
        img = image.img_to_array(img)
        # appending the image to the train_image list
        train_image_temp.append(img)
        #print(len(train_image_temp))
    #train_image.append(train_image_temp)
    X.append(np.array(train_image_temp,'float16'))
#print(train_image[2])
#train_image.dtype = 'float16'
# converting the list to numpy array
#X = np.array(train_image)
X_train = np.array(X,dtype='float16')
# shape of the array
X_train.shape

In [ ]:
print(len(X_train))

In [ ]:
label = np.zeros(len(X_train),dtype = int)
count = 5
for i in tqdm(range(len(X_train))):
    if (i == count and i < 48):
        label[i] = 1
        count += 6
    if  (i == 48 or i == 54):
        label[i] = 1
    if (i > 59):
        label[i] = 1
print(label)

In [ ]:
'''
label = np.zeros(len(X_train),dtype = int)
count = 40
for i in tqdm(range(len(X_train))):
    if (i == count):
        label[i] = 1
        count += 60
#print(label[101])

num_1 = train['image']
print(num_1)
label = np.zeros(len(X_train),dtype = int)
for i in tqdm(range(len(X_train))):
    num = num_1[i].split('\\',5)[5]
    # creating the image name
    #print(images[i].split('_rgb')[0].split('A')[1])
    if (num[i].split('_rgb')[0].split('A')[1] == '041'):
        label.append('Cough')
    # creating the class of image
    else:
        label.append('Others')
'''

In [ ]:
# creating the training and validation set
X_train = X_train.astype('float32')
X_train -= np.mean(X_train)
X_train /=np.max(X_train)

label = np_utils.to_categorical(label, 2)
X_train, X_test, y_train, y_test = train_test_split(X_train, label, random_state=42, test_size=0.2, stratify = label)

In [ ]:
#print(y_train, y_test)

In [ ]:
model = Sequential()
model.add(Convolution3D(32, 5, 5, 5, padding='same', name='conv1', dim_ordering='tf', input_shape=(15,112,112,3), activation='relu'))

model.add(MaxPooling3D(pool_size=(3, 3, 3)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, init='normal', activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(2,init='normal'))

model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, validation_data=(X_test,y_test),
          batch_size = 16, epochs = 15, shuffle=True)


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=32)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

In [ ]:
print(score)

In [ ]:
'''
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accrucy']
val_acc=hist.history['val_acc']
xc=range(100)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
'''

In [ ]:
# open the .txt file which have names of test videos
f = open(r"D:\2020Spring\636_project\data\test_list.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:]
#print(test)

In [ ]:
# storing the frames from test videos
Test_array = []
for i in tqdm(range(test.shape[0])):
#for i in range(1):
    test_image_temp = []
    count = 0
    videoFile = test['video_name'][i]
    cap = cv2.VideoCapture(r'D:\2020Spring\636_project\data\test_data1/' + videoFile)   # capturing the video from the given path
    frameCount = cap.get(7) #frame counts
    x = math.floor(frameCount / 15)
    #print([frameCount,x])
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (count == 15):
            break
        if (frameId == (x * count)):
            # storing the frames in a new folder named train_1
            frame=cv2.resize(frame,(112,112),interpolation=cv2.INTER_AREA)
            #filename =r'D:\2020Spring\636_project\data\test_1/' + videoFile +"_frame%d.jpg" % count;count+=1
            #cv2.imwrite(filename, frame)
            frame = image.img_to_array(frame)
            # appending the image to the train_image list
            test_image_temp.append(frame)
            #print(len(test_image_temp))
            count+=1
    cap.release()
    Test_array.append(np.array(test_image_temp,dtype='float16'))
Test_array_f = np.array(Test_array,dtype='float16')
Test_array_f = Test_array_f.astype('float32')
Test_array_f -= np.mean(Test_array_f)
Test_array_f /=np.max(Test_array_f)
#Test_array_f[0][:][:][:][:] = Test_array[:][:][:][:]
print(Test_array_f.shape)
#make prediction
prediction = model.predict_classes(Test_array_f)
print(prediction)